In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import sys
sys.path.append('..')
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

从零实现


In [2]:
#初始化模型参数
num_inputs,num_hiddens,num_outputs = vocab_size,256,vocab_size
print('use',device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0,0.01,size=shape),device=device,dtype=torch.float32)
        return torch.nn.Parameter(ts,requires_grad=True)
    def _three():
        return (_one((num_inputs,num_hiddens)),
                _one((num_hiddens,num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens,device=device,dtype=torch.float32),requires_grad=True))


    W_xz,W_hz,b_z = _three() #更新门参数
    W_xr,W_hr,b_r = _three()#重置门参数
    W_xh,W_hh,b_h = _three()#候选隐藏状态参数

    #输出层参数
    W_hq = _one((num_hiddens,num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs,device=device,dtype=torch.float32),requires_grad=True)

    return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q])

use cuda


In [3]:
#定义模型
#初始化隐藏状态
def init_gru_state(batch_size,num_hiddens,device):
    return (torch.zeros((batch_size,num_hiddens),device=device))

In [8]:
def gru(inputs,state,params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H= state
    outputs = []
    for x in inputs:
        Z = torch.sigmoid(torch.matmul(x,W_xz)+torch.matmul(H,W_hz) + b_z)
        R = torch.sigmoid(torch.matmul(x,W_xr)+torch.matmul(H,W_hr)+b_r)
        H_tilda = torch.tanh(torch.matmul(x,W_xh) + torch.matmul(R*H,W_hh) +b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, H

In [9]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

In [10]:
d2l.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, True, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

epoch 40, perplexity 145.580651, time 0.26 sec
 - 分开 我不要你的爱爱人  哼在在我的可爱人  哼在在在我的可爱人  坏的让我的爱人  哼在在我的可爱人 
 - 不分开 我想你的让我 我想你的让我 我想你的让我 我想你的让我 我想你的让我 我想你的让我 我想你的让我 
epoch 80, perplexity 33.298750, time 0.23 sec
 - 分开 我不要再想 我不能再想你 我不能再想你 我不能再想你 我不能再想你 我不能再想你 我不能再想你 我
 - 不分开 不知不觉 我不能再想 我不能再想你 我不能再想你 我不能再想你 我不能再想你 我不能再想你 我不能
epoch 120, perplexity 7.189187, time 0.23 sec
 - 分开不能再亚 我不要再想 我不要再想 我不 我不 我不能 爱情走的太快就像龙卷风 不能承受我已无处可躲 
 - 不分开远 我没你这的微笑每天 想要和你牵着我妈妈 难道你手不会再想 我不要再想 我不要再想 我不 我不 我
epoch 160, perplexity 2.427665, time 0.23 sec
 - 分开不住米亚平原 用楔形文字刻下了永远 脸已风化千年的誓言 一切又重演 爱在西元前 爱在西元前 周杰伦 
 - 不分开吗 然没有你去 我不要再想 我的好空 是静经风 我家定好生 我不好再想 我不要再想 我不要再想 我不


简洁实现

In [11]:
lr = 1e-2 # 注意调整学习率
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = d2l.RNNModel(gru_layer, vocab_size).to(device)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.013498, time 0.04 sec
 - 分开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让
 - 不分开不觉 我该好好生活 我该好好生活 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一
epoch 80, perplexity 1.009869, time 0.04 sec
 - 分开的玩笑 想通 却又再考倒我 说散 你想很久了吧? 我不想拆穿你 当作 是你开的玩笑 想通 却又再考倒
 - 不分开不起 广场一枚铜币 悲伤得很隐密 它在许愿池里轻轻叹息 太多的我爱你 让它喘不过气 已经 失去意义 
epoch 120, perplexity 1.007596, time 0.04 sec
 - 分开的爱写在西元前 深埋在美索不达米亚平原 几十个世纪后出土发现 泥板上的字迹依然清晰可见 我给你的爱写
 - 不分开不起 广场一枚铜币 悲伤得很隐密 它在许愿池里轻轻叹息 太多的我爱你 让它喘不过气 已经 失去意义 
epoch 160, perplexity 1.006594, time 0.04 sec
 - 分开的爱写在西元前 深埋在美索不达米亚平原 几十个世纪后出土发现 泥板上的字迹依然清晰可见 我给你的爱写
 - 不分开不起 广场一枚铜币 悲伤得很隐密 它在许愿池里轻轻叹息 太多的我爱你 让它喘不过气 已经 失去意义 
